In [1]:
import numpy as np
from bs4 import BeautifulSoup
import urllib3
import multiprocessing
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import json

In [2]:
client = MongoClient()
db = client['dota']

In [3]:
pro_matches_id_coll = db.pro_matches_id

In [4]:
def get_working_proxy():
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    proxy_list_site = 'https://free-proxy-list.net/'
    pm = urllib3.PoolManager(1, 
                             headers=user_agent, 
                             cert_reqs='CERT_REQUIRED')
    prx_req = pm.request('GET', proxy_list_site)
    prx_soup = BeautifulSoup(prx_req.data, 'html.parser')
    proxies_list = []
    for tr in prx_soup.find_all('tr'):
        tmp = []
        for td in tr.find_all('td')[:2]:
            tmp.append(td.text)
        if len(tmp) < 2:
            continue
        proxies_list.append([str(tmp[0]) + ':' + tmp[1]])
    
    proxies_list = proxies_list[:20]
    np.random.shuffle(proxies_list)
    for i, prx in enumerate(proxies_list):
        try:
            print('Getting working proxy...')
            cur_prx_address = 'https://' + str(prx[0]) + '/'
            prx_http = urllib3.ProxyManager(cur_prx_address,
                                            maxsize=1, 
                                            headers=user_agent, 
                                            cert_reqs='CERT_REQUIRED')
            r = prx_http.request('GET', 'https://stackoverflow.com/', timeout=0.7)
            if r.status == 200:
                print('Proxy found.')
                return cur_prx_address
            time.sleep(0.1)
        except Exception as err:
            # print(err)
            continue

In [5]:
def api_pro_matches_id(prx_address, start_id=9994966018):
    url = 'https://api.opendota.com/api/proMatches?less_than_match_id='
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    cur_url = url + str(start_id)
    prx_m = urllib3.ProxyManager(prx_address, headers=user_agent, cert_reqs='CERT_REQUIRED')
    r = prx_m.request('GET', cur_url, timeout=1)
    matches = json.loads(r.data)
    return matches

In [ ]:
last_id = 9994966018
cur_prx = get_working_proxy()
for i in range(1000):
    total_matches = pro_matches_id_coll.count()
    print(i, total_matches, last_id) if i % 10 == 0 else None
    try:
        cur_matches = api_pro_matches_id(cur_prx, start_id=last_id)
        ids = [x['match_id'] for x in cur_matches]
        if hasattr(cur_matches, 'error') and cur_matches['error'] == 'rate limit exceeded':
            cur_prx = get_working_proxy()
            continue
        last_id = np.min(ids)
        pro_matches_id_coll.insert(cur_matches)
        total_matches = pro_matches_id_coll.count()
    except DuplicateKeyError as de:
        continue
    except Exception as err:
        # raise
        print(err)
        continue

Getting working proxy...
Getting working proxy...
Proxy found.
0 55929 9994966018


/home/nemo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


10 55931 3842787669
20 55931 3778426484
30 55931 3687092089
40 55931 3620796674
50 55931 3551813576
60 55931 3471482236
70 55931 3406641735
80 55931 3282300073
90 55931 3202378832
100 55931 3155358972
110 55931 3083179978
120 55931 3031011614
130 55931 2920590237
140 55931 2836411803
150 55931 2762405085
160 55931 2707751660
170 55931 2647999621
180 55931 2527350837
190 55931 2469889797
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
200 55931 2447397655
string indices must be integers
210 55931 2390400509
220 55931 2346655729
230 55931 2276142931
240 55931 2198989614
250 55931 2095244332
string indices must be integers
string indices must be integers
string indices must be integers
260 55931 2021603155
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices 